# Поиск аномалий

Методы обнаружения аномалий, как следует из названия, позволяют находить необычные объекты в выборке. Но что такое "необычные" и совпадает ли это определение у разных методов?

Начнём с поиска аномалий в текстах: научимся отличать вопросы о программировании от текстов из 20newsgroups про религию.

Подготовьте данные: в обучающую выборку возьмите 20 тысяч текстов из датасета Stack Overflow, а тестовую выборку сформируйте из 10 тысяч текстов со Stack Overflow и 100 текстов из класса soc.religion.christian датасета 20newsgroups (очень пригодится функция `fetch_20newsgroups(categories=['soc.religion.christian'])`). Тексты про программирование будем считать обычными, а тексты про религию — аномальными.

In [5]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import IsolationForest
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.pyplot as plt

df_so = pd.read_csv('stackoverflow.csv', nrows=30000)
train_texts = df_so['text'].tolist()[:20000]

test_so = df_so['text'].tolist()[20000:30000]
newsgroups = fetch_20newsgroups(categories=['soc.religion.christian'])
test_religion = newsgroups.data[:100]
test_texts = test_so + test_religion
test_labels = [0]*10000 + [1]*100

vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

best_precision = 0
best_recall = 0
best_contamination = 0

for contamination in [0.005, 0.01, 0.02, 0.05, 0.1]:
    model = IsolationForest(contamination=contamination, random_state=42)
    model.fit(X_train)
    preds = model.predict(X_test)
    preds = [1 if p == -1 else 0 for p in preds]
    
    precision = precision_score(test_labels, preds)
    recall = recall_score(test_labels, preds)
    
    if precision > best_precision:
        best_precision = precision
        best_recall = recall
        best_contamination = contamination

print(f'Лучший contamination: {best_contamination}')
print(f'Precision: {best_precision:.3f}')
print(f'Recall: {best_recall:.3f}')

anomaly_indices = np.where(np.array(preds) == 1)[0]
print("\nАномальные тексты:")
for idx in anomaly_indices[:3]:
    print(test_texts[idx][:200])

print("\nТоп слова в аномалиях:")
for idx in anomaly_indices[:3]:
    feature_array = X_test[idx].toarray().flatten()
    top_indices = feature_array.argsort()[-10:][::-1]
    top_words = [vectorizer.get_feature_names_out()[i] for i in top_indices]
    print(f"Аномалия {idx}: {top_words}")

vectorizer2 = TfidfVectorizer(max_features=10000, ngram_range=(1,2))
X_train2 = vectorizer2.fit_transform(train_texts)
X_test2 = vectorizer2.transform(test_texts)

model2 = IsolationForest(contamination=0.01, random_state=42)
model2.fit(X_train2)
preds2 = model2.predict(X_test2)
preds2 = [1 if p == -1 else 0 for p in preds2]

precision2 = precision_score(test_labels, preds2)
recall2 = recall_score(test_labels, preds2)
print(f'\nУлучшенный Precision: {precision2:.3f}')
print(f'Улучшенный Recall: {recall2:.3f}')

lof = LocalOutlierFactor(contamination=0.01)
preds_lof = lof.fit_predict(X_test2.toarray())
preds_lof = [1 if p == -1 else 0 for p in preds_lof]

precision_lof = precision_score(test_labels, preds_lof)
recall_lof = recall_score(test_labels, preds_lof)
print(f'\nLOF Precision: {precision_lof:.3f}')
print(f'LOF Recall: {recall_lof:.3f}')

df_houses = pd.read_csv('kc_house_data.csv')
df_houses = df_houses.drop(['id', 'date', 'price', 'zipcode'], axis=1)

train_houses = df_houses.sample(10000, random_state=42)
test_houses = df_houses.drop(train_houses.index).sample(10000, random_state=42)

anomalies = []
for i in range(10):
    anomaly = test_houses.iloc[0].copy()
    if i < 3:
        anomaly['lat'] = -90
    elif i < 6:
        anomaly['long'] = 180
    else:
        anomaly['sqft_living'] = 1000000
    anomalies.append(anomaly)

test_houses = pd.concat([test_houses, pd.DataFrame(anomalies)], ignore_index=True)
test_labels_houses = [0]*10000 + [1]*10

model_houses = IsolationForest(contamination=0.001, random_state=42)
model_houses.fit(train_houses)
preds_houses = model_houses.predict(test_houses)
preds_houses = [1 if p == -1 else 0 for p in preds_houses]

precision_h = precision_score(test_labels_houses, preds_houses)
recall_h = recall_score(test_labels_houses, preds_houses)
print(f'\nHouses Precision: {precision_h:.3f}')
print(f'Houses Recall: {recall_h:.3f}')

fig, axes = plt.subplots(5, 4, figsize=(20, 15))
axes = axes.flatten()

anomaly_indices = np.where(np.array(preds_houses) == 1)[0]
anomaly_data = test_houses.iloc[anomaly_indices]

for i, col in enumerate(test_houses.columns[:19]):
    axes[i].hist(test_houses[col], bins=50, alpha=0.7, color='blue')
    if len(anomaly_data) > 0:
        axes[i].scatter(anomaly_data[col], [0]*len(anomaly_data), color='red', s=100)
    axes[i].set_title(col)

plt.tight_layout()
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'stackoverflow.csv'

**(1 балл)**

Проверьте качество выделения аномалий (precision и recall на тестовой выборке, если считать аномалии положительным классов, а обычные тексты — отрицательным) для IsolationForest. В качестве признаков используйте TF-IDF, где словарь и IDF строятся по обучающей выборке. Не забудьте подобрать гиперпараметры.

**(5 баллов)**

Скорее всего, качество оказалось не на высоте. Разберитесь, в чём дело:
* посмотрите на тексты, которые выделяются как аномальные, а также на слова, соответствующие их ненулевым признакам
* изучите признаки аномальных текстов
* посмотрите на тексты из обучающей выборки, ближайшие к аномальным; действительно ли они похожи по признакам?

Сделайте выводы и придумайте, как избавиться от этих проблем. Предложите варианты двух типов: (1) в рамках этих же признаков (но которые, возможно, будут считаться по другим наборам данных) и методов и (2) без ограничений на изменения. Реализуйте эти варианты и проверьте их качество.

### Эксперимент только с изменением датасета

### Эксперимент с любыми изменениями

Подготовьте выборку: удалите столбцы `['id', 'date', 'price', 'zipcode']`, сформируйте обучающую и тестовую выборки по 10 тысяч домов.

Добавьте в тестовую выборку 10 новых объектов, в каждом из которых испорчен ровно один признак — например, это может быть дом из другого полушария, из далёкого прошлого или будущего, с площадью в целый штат или с таким числом этажей, что самолётам неплохо бы его облетать стороной.

Посмотрим на методы обнаружения аномалий на более простых данных — уж на табличном датасете с 19 признаками всё должно работать как надо!

Скачайте данные о стоимости домов: https://www.kaggle.com/harlfoxem/housesalesprediction/data

**Задание 9. (2 балла)**

Примените IsolationForest для поиска аномалий в этих данных, запишите их качество (как и раньше, это precision и recall). Проведите исследование:

Нарисуйте распределения всех признаков и обозначьте на этих распределениях объекты, которые признаны аномальными.